In [1]:
import geopandas as gp
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

import optuna
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np
import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score

import os

from hyperparam_opt import *

In [2]:
pwd


'/mnt/c/Users/2toth/Desktop/546/project/2022-Project-Repo/serf'

In [3]:
ls

 Complete.csv*
 Forecasting.ipynb*
 Locations.csv*
 Main_SERF_nb.ipynb*
'Multivariate Correlation.ipynb'*
 NASA/
 Plotly.ipynb*
 SARIMAX-EXOG.ipynb*
 SERF-model-walkthru.ipynb*
 SERF.ipynb*
 SERF_environment.yml*
'Testing my patience good lord.ipynb'*
 Train-test-post-optimization.ipynb*
 Untitled.ipynb*
 __init__.py*
 __pycache__/
 arima_dataframe.py*
 arima_yearly_averages.py*
 forecast_single.py*
'hyperparam all files.ipynb'*
'hyperparam all files_2.ipynb'*
 hyperparam_opt.py*
 hyperparam_opt_2.py*
 hyperparam_opt_guide.ipynb*
 json_to_csv.py*
 model-accuracy.csv*
 model_testing_function.py*
 model_validation.py*
 not-the-actual-model.ipynb*
 sample-expanding-window-validation.ipynb*
'serf logo.png'*
 testing-all-samples.ipynb*
 tests/
 ts_train_test_split.py*
 unittesting-testing-grounds.ipynb*
 write_opt_hyperparam_to_location.ipynb*


In [11]:
arr = os.listdir('../../hyperparameters_2/')

arr2 = np.argsort(arr)
arr
#arr = arr.sort()
#arr.sort(key=lambda f: int(filter(str.isdigit, f)))
#arr = sorted(arr)
print(os.path.splitext(arr[1])[0][31:])

def filenumber(x):
    x = os.path.splitext(x)[0]
    return(int(x[31:]))

arr = sorted(arr, key = filenumber) 
#print(int(os.path.splitext(arr[4])[0].split('SARIMAX_hyperparameter_results_')[1]))
#arr = sorted(arr,key=lambda x: int(os.path.splitext(arr[x])[0].split('SARIMAX_hyperparameter_results_')[1]))
arr

1


['SARIMAX_hyperparameter_results_0.csv',
 'SARIMAX_hyperparameter_results_1.csv',
 'SARIMAX_hyperparameter_results_2.csv',
 'SARIMAX_hyperparameter_results_3.csv',
 'SARIMAX_hyperparameter_results_4.csv',
 'SARIMAX_hyperparameter_results_5.csv',
 'SARIMAX_hyperparameter_results_6.csv',
 'SARIMAX_hyperparameter_results_7.csv',
 'SARIMAX_hyperparameter_results_8.csv',
 'SARIMAX_hyperparameter_results_9.csv',
 'SARIMAX_hyperparameter_results_10.csv',
 'SARIMAX_hyperparameter_results_11.csv',
 'SARIMAX_hyperparameter_results_12.csv',
 'SARIMAX_hyperparameter_results_13.csv',
 'SARIMAX_hyperparameter_results_14.csv',
 'SARIMAX_hyperparameter_results_15.csv',
 'SARIMAX_hyperparameter_results_16.csv',
 'SARIMAX_hyperparameter_results_17.csv',
 'SARIMAX_hyperparameter_results_18.csv',
 'SARIMAX_hyperparameter_results_19.csv',
 'SARIMAX_hyperparameter_results_20.csv',
 'SARIMAX_hyperparameter_results_21.csv',
 'SARIMAX_hyperparameter_results_22.csv',
 'SARIMAX_hyperparameter_results_23.csv',
 '

In [12]:
for k in range(len(arr)):
    arr[k] = '../../hyperparameters_2/'+ arr[k]

In [13]:
arr

['../../hyperparameters_2/SARIMAX_hyperparameter_results_0.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_1.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_2.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_3.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_4.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_5.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_6.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_7.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_8.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_9.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_10.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_11.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_12.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_13.csv',
 '../../hyperparameters_2/SARIMAX_hyperparameter_results_14.csv',
 '../../hyperparamet

In [14]:
df = pd.read_csv(arr[0])
df = df.sort_values('value',ascending=False)
df.head(1)

,Unnamed: 0,number,value,datetime_start,datetime_complete,duration,params_D,params_P,params_Q,params_d,params_p,params_q,state
49,49,49,0.584455,2022-03-13 15:46:58.913611,2022-03-13 15:47:04.610650,0 days 00:00:05.697039,1,2,1,0,1,8,COMPLETE


,lat,lon,Place,p
0,49.0362,-122.3247,"Abbotsford, Canada",
1,49.0362,-118.4245,"Grand Forks, Canada",
2,48.9289,-117.8306,"Northport, WA",
3,48.7835,-122.3357,"Dewey, WA",
4,48.7826,-121.7994,"Mt. Baker, WA",
5,48.5767,-122.9509,"Shaw Island, WA",
6,48.5085,-117.3252,"Rudy, WA",
7,48.4093,-123.5716,"Metchosin, Canada",
8,48.0139,-123.4453,"Port Angeles, WA",
9,47.9735,-122.2038,"Everett, WA",


In [18]:
hyperparam=[]

for k in range(len(arr)):
    df = pd.read_csv(arr[k])
    df = df.sort_values('value',ascending=False)
    hyperparam.append(df[['params_D','params_P','params_Q','params_d','params_p','params_q']].head(1).values[0])
hyperparam = np.array(hyperparam)
hyperparam = np.array(hyperparam)
hyperparam

array([[1, 2, 1, 0, 1, 8],
       [0, 1, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 2],
       [1, 0, 1, 0, 7, 5],
       [1, 0, 1, 0, 4, 6],
       [1, 0, 1, 0, 7, 5],
       [0, 1, 2, 0, 8, 6],
       [1, 0, 1, 0, 7, 5],
       [1, 0, 1, 0, 2, 1],
       [1, 1, 1, 0, 7, 8],
       [1, 0, 1, 0, 7, 1],
       [1, 0, 1, 0, 8, 2],
       [1, 0, 2, 0, 8, 1],
       [1, 0, 1, 0, 1, 8],
       [1, 2, 1, 0, 8, 8],
       [1, 2, 2, 0, 6, 2],
       [0, 3, 1, 0, 8, 7],
       [1, 0, 2, 0, 8, 2],
       [1, 2, 3, 0, 3, 8],
       [1, 3, 3, 0, 3, 7],
       [1, 2, 2, 0, 3, 8],
       [1, 0, 2, 0, 6, 3],
       [1, 1, 2, 0, 7, 2],
       [1, 2, 1, 0, 8, 3],
       [1, 1, 2, 0, 6, 8],
       [0, 3, 2, 0, 7, 8],
       [1, 0, 1, 0, 3, 8]])

In [19]:
dfloc = pd.read_csv('../../Locations.csv')
dfloc['p'] = hyperparam[:,4]
dfloc['d'] = hyperparam[:,3]
dfloc['q'] = hyperparam[:,5]
dfloc['P'] = hyperparam[:,1]
dfloc['D'] = hyperparam[:,0]
dfloc['Q'] = hyperparam[:,2]
dfloc

,Unnamed: 0,lat,lon,Place,p,d,q,P,D,Q,filepath
0,0,49.0362,-122.3247,"Abbotsford, Canada",1,0,8,2,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
1,1,49.0362,-118.4245,"Grand Forks, Canada",1,0,0,1,0,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
2,2,48.9289,-117.8306,"Northport, WA",1,0,2,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
3,3,48.7835,-122.3357,"Dewey, WA",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
4,4,48.7826,-121.7994,"Mt. Baker, WA",4,0,6,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
5,5,48.5767,-122.9509,"Shaw Island, WA",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
6,6,48.5085,-117.3252,"Rudy, WA",8,0,6,1,0,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
7,7,48.4093,-123.5716,"Metchosin, Canada",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
8,8,48.0139,-123.4453,"Port Angeles, WA",2,0,1,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
9,9,47.9735,-122.2038,"Everett, WA",7,0,8,1,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...


In [130]:
dfloc.to_csv('../../hyperparameters_2/locations_2.csv')